In [ ]:
from __future__ import division

In [ ]:
import os
import random
import math
import datetime
import time

In [ ]:
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


In [5]:
from training import *

In [6]:
#def load_data(prefix):
#    features = pd.read_csv('{}/features.csv'.format(prefix),
#                       names=['game_idx', 'turn_idx'] + range(42),
#                       dtype='int')
#    
#    targets = pd.read_csv('{}/targets.csv'.format(prefix),
#                      names=['game_idx', 'turn_idx', 'win', 'lose', 'draw'],
#                      dtype='int')
#    
#    # Split into training and holdout,
#    # ensuring separate games 
#
#    train_games = pd.Series(features.game_idx.unique()).sample(frac=0.8).values #games = #features.game_idx.unique()
#
#    features_train = features[features.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)
#    targets_train = targets[targets.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)
#
#    tidx = list(features_train.index)
#    random.shuffle(tidx)
#    features_train = features_train.loc[tidx]
#    targets_train = targets_train.loc[tidx]
#
#    features_test = features[~features.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)
#    targets_test = targets[~targets.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)
#   
#    
#    return features_train, targets_train, features_test, targets_test

In [7]:
#def get_batch_iter(batch_size, batch_idx, dfs):
#    
#    length = len(dfs[0])
#    for df in dfs:
#        assert len(df) == length
#        
#    batches_per_df = int(math.ceil(length / batch_size))
#    
#    local_idx = batch_idx % batches_per_df
#    
#    start = local_idx*batch_size
#    end = (local_idx+1)*batch_size
#    
#    return [df.iloc[start:end] for df in dfs]
#
#
#def get_batch_random(batch_size, _, dfs):    
#    mask = pd.Series(dfs[0].index).sample(n=batch_size, replace=False)    
#    return [df.loc[mask] for df in dfs]
#
#def get_batch(batch_size, batch_idx, dfs, how='iter'):
#    if how == 'iter':
#        return get_batch_iter(batch_size, batch_idx, dfs)
#    elif how == 'random':
#        return get_batch_random(batch_size, batch_idx, dfs)
#    else:
#        raise Exception()

In [8]:
def run(graph, output_prefix, batch_size, num_batches, batch_how='iter'):
    
    with tf.Session(graph=graph) as sess:
    
        K.set_session(sess)
        sess.run(init_op)
    
        #suffix = datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
        #name = output_prefix #'{}_{}'.format(output_prefix, suffix)
        #save_name = "{}/model".format(name) #, suffix)

        train_writer = tf.summary.FileWriter(output_prefix, sess.graph)
        
        print "Running {}".format(output_prefix)
            
        t = time.time()
        delta_t = 0
        
        for i in range(num_batches):
        
            if i % 1000 == 0:
                
                delta_t = time.time() - t
                t = time.time()
                    
                acc = acc_value.eval(feed_dict={board: features_test, outcome: targets_test}).mean()
                print "Batch {} Hold-Out Accuracy: {} Time taken: {:.1f}s".format(i, acc, delta_t)

                summary = sess.run(all_summaries, feed_dict={board: features_test, outcome: targets_test})
                train_writer.add_summary(summary, i)            
            
            batch = get_batch(batch_size, i, [features_train, targets_train], how=batch_how)        
            train_step.run(feed_dict={board: batch[0], outcome: batch[1]})
        
        print "DONE TRAINING"
        print "FINAL ACCURACY: {}".format(acc)
        train_writer.close()
        
        model_dir = '{}/model'.format(output_prefix)
        print "SAVING TO: {}".format(model_dir)
        tf.train.Saver().save(sess, model_dir)

# Gen 1

In [9]:
# 100,000 rows of random vs random (gen 0) data
key = 'random-2017-10-21-13:41:47'

# 5000 rows of covnet vs covnet (gen 1) data
#key = 'gen-1-2017-10-22-133109'

In [10]:
#features = pd.read_csv('training_data/{}/features.csv'.format(key),
#                       names=['game_idx', 'turn_idx'] + range(42),
#                       dtype='int')
#
#targets = pd.read_csv('training_data/{}/targets.csv'.format(key),
#                      names=['game_idx', 'turn_idx', 'win', 'lose', 'draw'],
#                      dtype='int')

In [11]:
# Split into training and holdout,
# ensuring separate games 

#train_games = pd.Series(features.game_idx.unique()).sample(frac=0.8).values #games = #features.game_idx.unique()

#features_train = features[features.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)
#targets_train = targets[targets.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)

#tidx = list(features_train.index)
#random.shuffle(tidx)
#features_train = features_train.loc[tidx]
#targets_train = targets_train.loc[tidx]

#features_test = features[~features.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)
#targets_test = targets[~targets.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)

In [12]:
features_train, targets_train, features_test, targets_test = load_data('training_data/{}'.format(key))

In [13]:
print len(features_train), len(features_test)
print len(targets_train), len(targets_test)

print targets_test.win.value_counts(normalize=True).loc[1]
print targets_test.lose.value_counts(normalize=True).loc[1]
print targets_test.draw.value_counts(normalize=True).loc[1]

1656083 413338
1656083 413338
0.499966129415
0.4734454611
0.0265884094857


## Attempt 1: Normal dense fully-connected neural network

In [14]:
from keras.layers import Dense
from keras.objectives import categorical_crossentropy
from keras.metrics import categorical_accuracy as accuracy
from keras.regularizers import l2

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():

    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board')
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')

    # Fully connected layers
    
    x = Dense(512,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(board)
    
    x = Dense(512,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)
   
    x = Dense(512,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)
    
    x = Dense(48,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)

    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
        
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)    
    
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [15]:
run(graph, './models/dense_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
    batch_size=200,
    num_batches=10000)

Running ./models/dense_2017_10_22_153235
Batch 0 Hold-Out Accuracy: 0.351310551167 Time taken: 0.0s
Batch 1000 Hold-Out Accuracy: 0.575231432915 Time taken: 27.3s
Batch 2000 Hold-Out Accuracy: 0.572209656239 Time taken: 30.2s
Batch 3000 Hold-Out Accuracy: 0.576791882515 Time taken: 30.9s
Batch 4000 Hold-Out Accuracy: 0.578538656235 Time taken: 31.7s
Batch 5000 Hold-Out Accuracy: 0.576641857624 Time taken: 31.7s
Batch 6000 Hold-Out Accuracy: 0.576624929905 Time taken: 31.7s
Batch 7000 Hold-Out Accuracy: 0.57958137989 Time taken: 31.5s
Batch 8000 Hold-Out Accuracy: 0.581828892231 Time taken: 32.2s
Batch 9000 Hold-Out Accuracy: 0.582842588425 Time taken: 32.1s
DONE TRAINING
FINAL ACCURACY: 0.582842588425
SAVING TO: ./models/dense_2017_10_22_153235/model


## Attempt 2: More complicated covnets

In [17]:
from keras.layers import Conv2D, Flatten
from keras.layers.core import Reshape
from keras.layers.merge import Concatenate

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():
    
    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board') 
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')    
    
    dense_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l2(0.01),
        bias_regularizer=regularizers.l2(0.01),    
    )
    
    d1 = Dense(512, **dense_args)(board)
        
    # The input data is [col0=[row_0, row_1, ...], col1=[row_0, row_1], ...]
    rs = Reshape((7, 6, 1), input_shape=(42,))(board)
    
    conv_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l2(0.01),
        input_shape=(7, 6, 1),
        padding='valid'
    )
    
    # We use a few parallel covents, that we combine in the end        
    ca = Flatten()(Conv2D(8, kernel_size=(5, 5), **conv_args)(rs))
    cb = Flatten()(Conv2D(8, kernel_size=(4, 4), **conv_args)(rs))
    cc = Flatten()(Conv2D(4, kernel_size=(1, 4), **conv_args)(rs))
    cd = Flatten()(Conv2D(4, kernel_size=(4, 1), **conv_args)(rs))

    merged = Concatenate()([d1, ca, cb, cc, cd])
    
    x = Dense(256,  **dense_args)(merged)         
    x = Dense(128,  **dense_args)(x)    
    x = Dense(12,   **dense_args)(x)
    
    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
    
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)
        
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)    
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [18]:
sess = run(graph,
           './models/gen1-cov2d_beta_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
           batch_size=500,
           num_batches=20000)

Running ./models/gen1-cov2d_beta_2017_10_22_155652
Batch 0 Hold-Out Accuracy: 0.433485418558 Time taken: 0.0s
Batch 1000 Hold-Out Accuracy: 0.602630317211 Time taken: 32.1s
Batch 2000 Hold-Out Accuracy: 0.611956775188 Time taken: 39.8s
Batch 3000 Hold-Out Accuracy: 0.616105973721 Time taken: 38.7s
Batch 4000 Hold-Out Accuracy: 0.620376050472 Time taken: 38.7s
Batch 5000 Hold-Out Accuracy: 0.621392190456 Time taken: 38.4s
Batch 6000 Hold-Out Accuracy: 0.621827661991 Time taken: 37.1s
Batch 7000 Hold-Out Accuracy: 0.620980918407 Time taken: 36.3s
Batch 8000 Hold-Out Accuracy: 0.621375262737 Time taken: 38.1s
Batch 9000 Hold-Out Accuracy: 0.620320439339 Time taken: 35.8s
Batch 10000 Hold-Out Accuracy: 0.620627641678 Time taken: 42.5s
Batch 11000 Hold-Out Accuracy: 0.620663940907 Time taken: 36.9s
Batch 12000 Hold-Out Accuracy: 0.62019944191 Time taken: 35.9s
Batch 13000 Hold-Out Accuracy: 0.618261575699 Time taken: 35.5s
Batch 14000 Hold-Out Accuracy: 0.618834972382 Time taken: 35.8s
Batc

In [16]:
# Gen 2

In [ ]:
# Use all the advanced data
# 10,000 rows of gen-1 vs gen-1 data
key = 'gen-1-cov2d_beta_2017_10_22_142925'
feat_train, target_train, feat_test, target_test = load_data('training_data/gen-1-cov2d_beta_2017_10_22_142925')